In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../modules/'))
sys.path.insert(1, os.path.join(sys.path[0], '../data_creation/'))
sys.path.insert(1, os.path.join(sys.path[0], '../networks/'))
sys.path.insert(1, os.path.join(sys.path[0], '../network_trainers/'))

from Data_Creator_C import Data_Creator_C as Data_Creator
from CNN_DS_BN_C import CNN_DS_BN_C
from CNN_C_Trainer import CNN_C_Trainer

from data_manipulation import *
import tensorflow as tf

In [2]:
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

In [3]:
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

In [12]:
network = CNN_DS_BN_C('CNN_DS_BN_C_161_blue_test', 4, 161)
network.adam_initial_learning_rate = 1e-05
network.print_params()

{'adam_initial_learning_rate': 1e-05,
 'log_dir': 'logs/',
 'name': 'CNN_DS_BN_C_161_blue_test',
 'num_classes': 161,
 'num_downsamples': 4}


In [13]:
trainer = CNN_C_Trainer(network,
                        Data_Creator,
                        161,
                        num_epochs = 100,
                        batch_size=2,
                        model_save_interval = 1)
trainer.print_params()

{'batch_size': 2,
 'downsample_keep_prob': 0.9,
 'log_dir': 'logs/',
 'metric_names': ['costs', 'accuracies'],
 'model_save_interval': 1,
 'name': 'CNN_DS_BN_C_161_blue_test',
 'num_classes': 161,
 'num_epochs': 100,
 'pretrained_model_path': None,
 'sample_keep_prob': 0.8}


In [14]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains,
                 num_flatnesses = 10,
                 precision = 0.00025)

In [15]:
trainer.train()

saving metrics - saved